# Sentiment Analysis Part 2

_Natural Langauge Processing Nanodegree Program_

---



## Step 5: Switching gears - RNNs

We just saw how the task of sentiment analysis can be solved via a traditional machine learning approach: BoW + a nonlinear classifier. We now switch gears and use Recurrent Neural Networks, and in particular LSTMs, to perform sentiment analysis in Keras. Conveniently, Keras has a built-in [IMDb movie reviews dataset](https://keras.io/datasets/#imdb-movie-reviews-sentiment-classification) that we can use, with the same vocabulary size.

In [1]:
from keras.datasets import imdb  # import the built-in imdb dataset in Keras

# Set the vocabulary size
vocabulary_size = 5000

# Load in training and test data (note the difference in convention compared to scikit-learn)
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocabulary_size)
print("Loaded dataset with {} training samples, {} test samples".format(len(X_train), len(X_test)))

Using TensorFlow backend.


17465344/17464789 [==============================] - 0s 0us/step
Loaded dataset with 25000 training samples, 25000 test samples


In [2]:
# Inspect a sample review and its label
print("--- Review ---")
print(X_train[7])
print("--- Label ---")
print(y_train[7])

--- Review ---
[1, 11, 14, 117, 22, 75, 28, 49, 87, 105, 21, 6, 55, 2012, 114, 12, 9, 165, 327, 8, 106, 88, 2, 127, 6, 87, 292, 33, 2, 4, 2, 4544, 5, 4, 2, 1519, 14, 166, 4, 529, 859, 4, 2, 1882, 15, 14, 20, 1505, 9, 18, 11, 192, 51, 75, 28, 133, 9, 32, 4, 614, 2, 5, 2118, 276, 23, 4065, 11, 6, 967, 1931, 74, 15, 7, 6, 1844, 407, 24, 6, 933, 20, 60, 48, 12, 494, 8, 30, 21, 128, 74, 35, 856, 1170, 20]
--- Label ---
1


Notice that the label is an integer (0 for negative, 1 for positive), and the review itself is stored as a sequence of integers. These are word IDs that have been preassigned to individual words. To map them back to the original words, you can use the dictionary returned by `imdb.get_word_index()`.

In [3]:
# Map word IDs back to words
word2id = imdb.get_word_index()
id2word = {i: word for word, i in word2id.items()}
print("--- Review (with words) ---")
print([id2word.get(i, " ") for i in X_train[7]])
print("--- Label ---")
print(y_train[7])

1646592/1641221 [==============================] - 0s 0us/step
--- Review (with words) ---
['the', 'this', 'as', 'over', 'you', 'bad', 'one', 'good', 'him', 'films', 'not', 'is', 'time', 'mrs', 'little', 'that', 'it', 'look', 'mind', 'in', 'character', 'most', 'and', 'end', 'is', 'him', 'together', 'they', 'and', 'of', 'and', 'square', 'to', 'of', 'and', 'race', 'as', 'find', 'of', 'self', 'poorly', 'of', 'and', 'finest', 'for', 'as', 'on', 'flaws', 'it', 'but', 'this', 'enough', 'when', 'bad', 'one', 'scene', 'it', 'an', 'of', 'shown', 'and', 'to', 'leader', 'maybe', 'are', 'neo', 'this', 'is', 'business', 'tarzan', 'been', 'for', 'br', 'is', 'plots', 'itself', 'his', 'is', 'beauty', 'on', 'which', 'what', 'that', 'tries', 'in', 'at', 'not', 'still', 'been', 'so', 'forget', "weren't", 'on']
--- Label ---
1


In [4]:
max([len(x) for x in X_train]), min([len(x) for x in X_train])

(2494, 11)

Unlike our Bag-of-Words approach, where we simply summarized the counts of each word in a document, this representation essentially retains the entire sequence of words (minus punctuation, stopwords, etc.). This is critical for RNNs to function. But it also means that now the features can be of different lengths!

#### Question: Variable length reviews

What is the maximum review length (in terms of number of words) in the training set? What is the minimum?

#### Answer:

Maximum number of words in a review is 2494. Minimum is 11.


### Pad sequences

In order to feed this data into your RNN, all input documents must have the same length. Let's limit the maximum review length to `max_words` by truncating longer reviews and padding shorter reviews with a null value (0). You can accomplish this easily using the [`pad_sequences()`](https://keras.io/preprocessing/sequence/#pad_sequences) function in Keras. For now, set `max_words` to 500.

In [5]:
from keras.preprocessing import sequence

# Set the maximum number of words per document (for both training and testing)
max_words = 500

# TODO: Pad sequences in X_train and X_test
X_train_padded = sequence.pad_sequences(X_train, maxlen=max_words)
X_test_padded = sequence.pad_sequences(X_test, maxlen=max_words)

### Design an RNN model for sentiment analysis

Build your model architecture in the code cell below. We have imported some layers from Keras that you might need but feel free to use any other layers / transformations you like.

Remember that your input is a sequence of words (technically, integer word IDs) of maximum length = `max_words`, and your output is a binary sentiment label (0 or 1).

In [6]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout

# TODO: Design your model
model = Sequential()
model.add(Embedding(vocabulary_size, 256, input_length=max_words))
model.add(LSTM(128))
model.add(Dropout(.5))
model.add(Dense(1, activation='sigmoid'))

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 256)          1280000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               197120    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 1,477,249
Trainable params: 1,477,249
Non-trainable params: 0
_________________________________________________________________
None


#### Question: Architecture and parameters

Briefly describe your neural net architecture. How many model parameters does it have that need to be trained?

#### Answer:

It's a simple default LSTM architecture from Keras reference. In total it has 1477249 parameters that have to be trained.

### Train and evaluate your model

Now you are ready to train your model. In Keras world, you first need to _compile_ your model by specifying the loss function and optimizer you want to use while training, as well as any evaluation metrics you'd like to measure. Specify the approprate parameters, including at least one metric `'accuracy'`.

In [7]:
# TODO: Compile your model, specifying a loss function, optimizer, and metrics
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Once compiled, you can kick off the training process. There are two important training parameters that you have to specify - **batch size** and **number of training epochs**, which together with your model architecture determine the total training time.

Training may take a while, so grab a cup of coffee, or better, go for a hike! If possible, consider using a GPU, as a single training run can take several hours on a CPU.

In [9]:
# TODO: Specify training parameters: batch size and number of epochs
batch_size = 32
num_epochs = 2

# TODO(optional): Reserve/specify some training data for validation (not to be used for training)
validation_split=.1
# TODO: Train your model
model.fit(X_train_padded, y_train, batch_size=batch_size, epochs=num_epochs, validation_split=validation_split)

Train on 22500 samples, validate on 2500 samples
Epoch 1/2
22500/22500 [==============================] - 1535s 68ms/step - loss: 0.4750 - acc: 0.7776 - val_loss: 0.4032 - val_acc: 0.8200
Epoch 2/2
22500/22500 [==============================] - 1527s 68ms/step - loss: 0.4609 - acc: 0.7848 - val_loss: 0.3355 - val_acc: 0.8564


In [12]:
import os

cache_dir = os.path.join("cache", "sentiment_analysis")
os.makedirs(cache_dir, exist_ok=True)

# Save your model, so that you can quickly load it in future (and perhaps resume training)
model_file = "rnn_model.h5"  # HDF5 file
model.save(os.path.join(cache_dir, model_file))

# Later you can load it using keras.models.load_model()
#from keras.models import load_model
#model = load_model(os.path.join(cache_dir, model_file))

Once you have trained your model, it's time to see how well it performs on unseen test data.

In [15]:
# Evaluate your model on the test set
scores = model.evaluate(X_test_padded, y_test, verbose=1)  # returns loss and other metrics specified in model.compile()
print("Test accuracy:", scores[1])  # scores[1] should correspond to accuracy if you passed in metrics=['accuracy']

25000/25000 [==============================] - 434s 17ms/step
Test accuracy: 0.85012
